In [79]:
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials
import sqlite3
import datetime

In [80]:
ticker_source = pd.read_html('https://topforeignstocks.com/listed-companies-lists/the-complete-list-of-listed-companies-in-singapore/')[0]
print(ticker_source)

     S.No.      Trading Name    Code                            Sector
0        1           3Cnergy  502.SI                       Real Estate
1        2      5E Resources  NLC.SI  Industrial & Commercial Services
2        3          8Telecom  AZG.SI              Technology Equipment
3        4                9R  1Y1.SI                  Industrial Goods
4        5               ABR  533.SI                  Food & Beverages
..     ...               ...     ...                               ...
649    650  Zheneng Jinjiang  BWM.SI                  Renewable Energy
650    651   Zhongmin Baihui  5SR.SI                         Retailers
651    652    Zhongxin Fruit  5EG.SI                  Food & Beverages
652    653        ZICO Hldgs  40W.SI     Banking & Investment Services
653    654             Zixin  42W.SI                  Food & Beverages

[654 rows x 4 columns]


In [81]:
example = yf.Ticker(ticker_source['Code'][0]).info
print(example)
print(example['quoteType'])
print(example['marketCap'])
print(example['averageVolume'])
first_trade_date = datetime.datetime.fromtimestamp(example['firstTradeDateEpochUtc']).strftime('%Y-%m-%d')
print(first_trade_date)

{'address1': '82 Ubi Avenue 4', 'address2': 'No. 05-04, Edward Boustead Centre', 'city': 'Singapore', 'zip': '408832', 'country': 'Singapore', 'phone': '65 6970 8117', 'website': 'https://www.3cnergy.com.sg', 'industry': 'Real Estate—Diversified', 'industryDisp': 'Real Estate—Diversified', 'sector': 'Real Estate', 'sectorDisp': 'Real Estate', 'longBusinessSummary': '3Cnergy Limited, an investment holding company, provides real estate and property development consultancy services in Singapore and Malaysia. It offers architectural design, project financial feasibility assessment, engineering expertise, and construction management services; and real estate valuation and appraisal services for housing and development board flats, private residential, commercial, and industrial properties for government agencies, financial institutions, corporations, and private individuals. The company was formerly known as HSR Global Limited and changed its name to 3Cnergy Limited in November 2013. 3Cnerg

## Filter out penny stocks
1. Stocks with a market cap above 10 million
2. Stocks with above 500k volume

In [83]:
from urllib.error import HTTPError

counter = 0
delisted = []
non_equity = []
insufficient_data = []
insufficient_market_cap = []

for idx, ticker in enumerate(ticker_source['Code']):
    
    try:
        data = yf.Ticker(ticker)
        quote_type = data.info['quoteType']
    except Exception as e:
        print(f"{ticker} is delisted")
        delisted.append(idx)
        continue
    
    if quote_type != 'EQUITY':
        print(f"{ticker} is not an equity. Data not retrieved.")
        non_equity.append(idx)
        continue
    
    if 'marketCap' not in data.info or 'averageVolume' not in data.info: 
        print(idx, ticker)
        print(f"{ticker} has insufficient data. Data not retrieved.")
        insufficient_data.append(idx)
        continue
    
    market_cap = data.info['marketCap']
    avg_volume = data.info['averageVolume']
    first_trade_date = datetime.datetime.fromtimestamp(example['firstTradeDateEpochUtc']).strftime('%Y-%m-%d')
    if market_cap < 10e6 or avg_volume < 500e3 or first_trade_date > '2010-01-01':
        print(f"{ticker} has insufficient market cap or average volume. Data not retrieved.")
        insufficient_market_cap.append(idx)       
    else:
        counter += 1
        
print(f"{counter} tickers have sufficient market cap and average volume.")

502.SI has insufficient market cap or average volume. Data not retrieved.
NLC.SI has insufficient market cap or average volume. Data not retrieved.
AZG.SI is not an equity. Data not retrieved.
1Y1.SI has insufficient market cap or average volume. Data not retrieved.
533.SI has insufficient market cap or average volume. Data not retrieved.
L5I.SI has insufficient market cap or average volume. Data not retrieved.
541.SI has insufficient market cap or average volume. Data not retrieved.
570.SI has insufficient market cap or average volume. Data not retrieved.
QZG.SI has insufficient market cap or average volume. Data not retrieved.
AYV.SI has insufficient market cap or average volume. Data not retrieved.
43F.SI has insufficient market cap or average volume. Data not retrieved.
BLZ.SI has insufficient market cap or average volume. Data not retrieved.
43Q.SI has insufficient market cap or average volume. Data not retrieved.
XVG.SI has insufficient market cap or average volume. Data not retr

In [ ]:
copy = ticker_source.copy()
print(f'{len(delisted)} delisted stocks')
print(f'{len(non_equity)} non-equity stocks')
print(f'{len(insufficient_data)} stocks with insufficient data')
print(f'{len(insufficient_market_cap)} stocks with insufficient market cap, volume or trade date after 2010-01-01')
to_drop = delisted + non_equity + insufficient_data + insufficient_market_cap
ticker_source.drop(to_drop, inplace=True)
print(f'{len(ticker_source)} stocks remaining')

151 stocks remaining


In [88]:
ticker_source.head()

,S.No.,Trading Name,Code,Sector
0,1,3Cnergy,502.SI,Real Estate
1,2,5E Resources,NLC.SI,Industrial & Commercial Services
2,3,8Telecom,AZG.SI,Technology Equipment
3,4,9R,1Y1.SI,Industrial Goods
4,5,ABR,533.SI,Food & Beverages


In [103]:
connection = sqlite3.connect('database.db')
cursor = connection.cursor()
cursor.execute('CREATE TABLE IF NOT EXISTS equities (id INTEGER PRIMARY KEY, name TEXT, ticker TEXT, sector TEXT)')
data = cursor.execute('SELECT * FROM equities').fetchall()

if data == []:
    for idx,stock in enumerate(ticker_source.iterrows()):
        cursor.execute(f'INSERT INTO equities VALUES ({idx}, "{stock[1][1]}", "{stock[1][2]}", "{stock[1][3]}")')
else:
    data = pd.DataFrame(data, columns=['id', 'name', 'ticker', 'sector'])
    print(data)
connection.commit()
connection.close()

      id              name  ticker                            sector
0      0           3Cnergy  502.SI                       Real Estate
1      1      5E Resources  NLC.SI  Industrial & Commercial Services
2      2          8Telecom  AZG.SI              Technology Equipment
3      3                9R  1Y1.SI                  Industrial Goods
4      4               ABR  533.SI                  Food & Beverages
..   ...               ...     ...                               ...
649  649  Zheneng Jinjiang  BWM.SI                  Renewable Energy
650  650   Zhongmin Baihui  5SR.SI                         Retailers
651  651    Zhongxin Fruit  5EG.SI                  Food & Beverages
652  652        ZICO Hldgs  40W.SI     Banking & Investment Services
653  653             Zixin  42W.SI                  Food & Beverages

[654 rows x 4 columns]
